In [56]:
from typing import List, final
from abc import ABC, abstractmethod
import csv
from collections import defaultdict, Counter
import math

In [57]:
class Figure(ABC):
    dimensions: dict
    calculated_figures: defaultdict = defaultdict(list)

    def __init__(self, **dimensions) -> None:
        self.dimensions = {key: int(value) for key, value in dimensions.items()}
        Figure.calculated_figures[self.__class__.__name__].append(self)

    @abstractmethod
    def get_area():
        pass

    @final
    def statistics():
        print('Figures count')
        for items in Figure.calculated_figures:
            figures = Figure.calculated_figures[items]
            print(f'{items}={len(figures)}')
            print(f'Max area = {max(figures)}')
            print(f'Min area = {min(figures)}')

    def __ge__(self, p_other: 'Figure') -> bool:
        return self.get_area() >= p_other.get_area()

    def __le__(self, p_other: 'Figure') -> bool:
        return self.get_area() <= p_other.get_area()

    def __gt__(self, p_other: 'Figure') -> bool:
        return self.get_area() > p_other.get_area()

    def __lt__(self, p_other: 'Figure') -> bool:
        return self.get_area() < p_other.get_area()

    def __eq__(self, p_other: 'Figure') -> bool:
        return self.get_area == p_other.get_area

    def __repr__(self) -> str:
        return f'[{self.__class__.__name__:<9}], area={self.get_area():<6.2f}, dimensions={self.dimensions}'

In [58]:
class Square(Figure):
    def __init__(self, **dimensions) -> None:
        super().__init__(**dimensions)

    def get_area(self):
        return self.dimensions['a'] ** 2

    def __repr__(self) -> str:
        return super().__repr__()

In [59]:
class Circle(Figure):
    def __init__(self, **dimensions) -> None:
        super().__init__(**dimensions)

    def get_area(self):
        return math.pi * self.dimensions['r'] ** 2

    def __repr__(self) -> str:
        return super().__repr__()

In [60]:
class Triangle(Figure):
    def __init__(self, **dimensions) -> None:
        super().__init__(**dimensions)

    def get_area(self):
        return (self.dimensions['a'] * self.dimensions['h']) / 2

    def __repr__(self) -> str:
        return super().__repr__()

In [61]:
class Trapezoid(Figure):
    def __init__(self, **dimensions) -> None:
        super().__init__(**dimensions)

    def get_area(self):
        return (self.dimensions['a'] + self.dimensions['b']) * self.dimensions['h'] / 2

    def __repr__(self) -> str:
        return super().__repr__()

In [62]:
with open('figures_schema.txt', newline='') as f:
    reader = csv.reader(f)
    read_lines = list(reader)[1:]
    figures_definition = {class_name: dimensions for class_name, *dimensions in read_lines}

with open('figures.txt', newline='') as f:
    reader = csv.reader(f)
    # take values only if already exist definition
    figures = [values for values in reader if values[0] in list(figures_definition.keys())]

In [63]:
figures, figures_definition

([['kwadrat', '3'],
  ['trojkat', '2', '5'],
  ['trapez', '10', '20', '3'],
  ['kolo', '10'],
  ['kolo', '15'],
  ['kwadrat', '6']],
 {'kwadrat': ['a'],
  'trojkat': ['a', 'h'],
  'trapez': ['a', 'b', 'h'],
  'kolo': ['r']})

In [64]:
mapped_figures = [(figure[0], dict(zip(figures_definition[figure[0]], figure[1:]))) for figure in figures]
mapped_figures

[('kwadrat', {'a': '3'}),
 ('trojkat', {'a': '2', 'h': '5'}),
 ('trapez', {'a': '10', 'b': '20', 'h': '3'}),
 ('kolo', {'r': '10'}),
 ('kolo', {'r': '15'}),
 ('kwadrat', {'a': '6'})]

In [65]:
prepared_figures = defaultdict(list)
for figure_name, dimensions in mapped_figures:
    prepared_figures[figure_name].append(dimensions)
prepared_figures

defaultdict(list,
            {'kwadrat': [{'a': '3'}, {'a': '6'}],
             'trojkat': [{'a': '2', 'h': '5'}],
             'trapez': [{'a': '10', 'b': '20', 'h': '3'}],
             'kolo': [{'r': '10'}, {'r': '15'}]})

In [66]:
mapped_types = {'kwadrat': Square, 'trojkat': Triangle, 'trapez': Trapezoid, 'kolo': Circle}
mapped_types

{'kwadrat': __main__.Square,
 'trojkat': __main__.Triangle,
 'trapez': __main__.Trapezoid,
 'kolo': __main__.Circle}

In [67]:
prepared_figures.items()

dict_items([('kwadrat', [{'a': '3'}, {'a': '6'}]), ('trojkat', [{'a': '2', 'h': '5'}]), ('trapez', [{'a': '10', 'b': '20', 'h': '3'}]), ('kolo', [{'r': '10'}, {'r': '15'}])])

In [68]:
for category, items in prepared_figures.items():
    print(f'Category={category}')
    category_func = mapped_types[category]
    for values in items:
        result = category_func(**values)
        print(result)

    # figure_type = list(set(_figure.values()) & set(_figure.keys()))[0]
    # _figure.pop(figure_type)
    # new_figure = mapped_types[figure_type](**_figure)
    # print(new_figure)

Category=kwadrat
[Square   ], area=9.00  , dimensions={'a': 3}
[Square   ], area=36.00 , dimensions={'a': 6}
Category=trojkat
[Triangle ], area=5.00  , dimensions={'a': 2, 'h': 5}
Category=trapez
[Trapezoid], area=45.00 , dimensions={'a': 10, 'b': 20, 'h': 3}
Category=kolo
[Circle   ], area=314.16, dimensions={'r': 10}
[Circle   ], area=706.86, dimensions={'r': 15}


In [69]:
Figure.calculated_figures

defaultdict(list,
            {'Square': [[Square   ], area=9.00  , dimensions={'a': 3},
              [Square   ], area=36.00 , dimensions={'a': 6}],
             'Triangle': [[Triangle ], area=5.00  , dimensions={'a': 2, 'h': 5}],
             'Trapezoid': [[Trapezoid], area=45.00 , dimensions={'a': 10, 'b': 20, 'h': 3}],
             'Circle': [[Circle   ], area=314.16, dimensions={'r': 10},
              [Circle   ], area=706.86, dimensions={'r': 15}]})

In [70]:
Figure.statistics()

Figures count
Square=2
Max area = [Square   ], area=36.00 , dimensions={'a': 6}
Min area = [Square   ], area=9.00  , dimensions={'a': 3}
Triangle=1
Max area = [Triangle ], area=5.00  , dimensions={'a': 2, 'h': 5}
Min area = [Triangle ], area=5.00  , dimensions={'a': 2, 'h': 5}
Trapezoid=1
Max area = [Trapezoid], area=45.00 , dimensions={'a': 10, 'b': 20, 'h': 3}
Min area = [Trapezoid], area=45.00 , dimensions={'a': 10, 'b': 20, 'h': 3}
Circle=2
Max area = [Circle   ], area=706.86, dimensions={'r': 15}
Min area = [Circle   ], area=314.16, dimensions={'r': 10}
